<a href="https://colab.research.google.com/github/lostb1d/LST/blob/main/LST_Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
#loading the Earth Engine API, GEE Authenticating and Loading Project
import ee
import folium
from IPython.display import Image
import geemap

try:
  ee.Initialize()
except Exception as e:
  ee.Authenticate()
ee.Authenticate()
ee.Initialize(project='lst-nou')


In [3]:
#loading the shapefile from the above mounted google drive

shape_path = '/content/drive/MyDrive/LST/Shapefiles/Dang/Bajura_Local.shp'
print(shape_path)
shp = geemap.shp_to_ee(shape_path)


/content/drive/MyDrive/LST/Shapefiles/Dang/Bajura_Local.shp


In [12]:
# Define a region of interest
roi = ee.Geometry.Polygon( [
            [
              82.21808522925141,
              28.336850451368647
            ],
            [
              81.91592954680112,
              28.126683552796777
            ],
            [
              81.90291805808266,
              27.841984062675692
            ],
            [
              82.443617700363,
              27.61804029507458
            ],
            [
              82.68939026503506,
              27.633411121765874
            ],
            [
              82.92215134098836,
              27.711512766230356
            ],
            [
              82.69328913525277,
              28.197409554609237
            ],
            [
              82.50245396738859,
              28.300565811038936
            ],
            [
              82.21808522925141,
              28.336850451368647
            ]
          ])

# Access the Landsat 8 imagery collection
image_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')

# Filter the collection by date
image = image_collection.filterDate('2015-01-01', '2015-12-31').filterBounds(roi).sort('CLOUD_COVER').first()
image = image.clip(roi)


landsat_rgb = image.select(['B4', 'B3', 'B2'])

# Display the image
print(image.getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [3355, 2648], 'origin': [2850, 506], 'crs': 'EPSG:32644', 'crs_transform': [30, 0, 503385, 0, -30, 3150315]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [3355, 2648], 'origin': [2850, 506], 'crs': 'EPSG:32644', 'crs_transform': [30, 0, 503385, 0, -30, 3150315]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [3355, 2648], 'origin': [2850, 506], 'crs': 'EPSG:32644', 'crs_transform': [30, 0, 503385, 0, -30, 3150315]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [3355, 2648], 'origin': [2850, 506], 'crs': 'EPSG:32644', 'crs_transform': [30, 0, 503385, 0, -30, 3150315]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, '

In [13]:
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}

def get_tile_url(ee_image, vis_params):
  map_id_dict = ee.Image(ee_image).getMapId(vis_params)
  map_id = map_id_dict['tile_fetcher'].url_format
  return map_id

tile_url = get_tile_url(landsat_rgb, vis_params)
print(tile_url)

https://earthengine.googleapis.com/v1/projects/lst-nou/maps/3210797abab4670ca22e544eb8dad4f1-0ca456b867c1631006c9df174169ae2b/tiles/{z}/{x}/{y}


In [15]:
# Create a Folium map object
map = folium.Map(location=[28.17, 82.37], zoom_start=10)

# Add the tile layer to the map
folium.TileLayer(
    tiles=tile_url,
    attr='Google Earth Engine',
    max_zoom=24
).add_to(map)

# Display the map
map
